In [ ]:
%pyspark
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_games_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True)
video_games_df.head()

In [ ]:
%pyspark
# Row Count
video_games_df.count()

In [ ]:
%pyspark
from pyspark.sql.functions import to_date
# Review DataFrame
review_id_table = video_games_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_table.head()

In [ ]:
%pyspark
products = video_games_df.select(["product_id", "product_title"]).drop_duplicates()
products.head()

In [ ]:
%pyspark
customers = video_games_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers.head()

In [ ]:
%pyspark
vine_table = video_games_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.head()

In [ ]:
%pyspark
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/big_data_db"
config = {"user":"root", "password": "<password>", "driver":"org.postgresql.Driver"}

In [ ]:
%pyspark
# Write review_id_df to table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
%pyspark
# Write products_df to table in RDS
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
%pyspark
# Write customers_df to table in RDS
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
%pyspark
# Write vine_df to table in RDS
vine_table.write.jdbc(url=jdbc_url, table='vines', mode=mode, properties=config)